In [ ]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np
import re
import io
from config import api_key_prism
from collections import OrderedDict
from pandas.io.json import json_normalize  
import time 

In [ ]:
## API request from Prism

url = 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/records'

# records = 100
# size = 100 - total

params={'size': 100, 
        'offset': 100,
#         'page': 2, 
        'status': 
        'published', 
        'sort': 'bestmatch', 
        'all_versions': 'false', 
        'access_token': api_key_prism
       }

response = requests.get(url, params=params)


print(response.url)
print(response.status_code)

response_json = response.json()['hits']
total = response.json()['hits']['total']

print(json.dumps(response_json, indent=4, sort_keys=True))

In [ ]:
len(response_json['hits'])

In [ ]:
# import requests

# ## Starting URL
# url = 'https://vtfsmghslrepo02.fsm.northwestern.edu/api/records'
# offset = 0
# size = 100
# records=[]


# while True: 
    
#     ## Python time method sleep() suspends execution for the given number of seconds 
#     time.sleep(0.1) 
    
#     print("------")
    
#     params={'size': size, 
#         'offset': offset,
# #         'page': 2, 
#         'status': 
#         'published', 
#         'sort': 'bestmatch', 
#         'all_versions': 'false', 
#         'access_token': api_key_prism
#        }

#     response = requests.get(url, params=params)
    
#     print(response.url)
#     print(response.status_code)
    
#     response_json = response.json()['hits']
# #     total = response.json()['hits']['total']

    
#     ## Did we find any records? 
#     if len(response_json["hits"]) == 0:
#         ## If no, then break and break the while loop.
#         break
    
#     ## If we did find records, add them to the list and move on to next offset
#     records.extend(response_json["hits"])
#     offset= offset + 100
   
# ## Pros: very flexible
# ## Cons: extra request, doesn't really listen to the API for the next URL

In [ ]:
records = response.json()['hits']
print(records)

In [ ]:
len(records)
print(type(records))

In [ ]:
# prism_item_df = pd.DataFrame.from_dict(json_normalize(response_json, meta=['hits']))
# prism_item_df.head()

prism_item_df = pd.json_normalize(records, max_level = 1)
prism_item_df.head()

In [ ]:
remove_hits_nest = prism_item_df.apply(lambda x: pd.Series(x['hits']),axis=1).stack().reset_index(level=1, drop=True).to_frame()
remove_hits_nest.rename(columns = {0:'data'}, inplace = True)
remove_hits_nest.head()


In [ ]:
def flatten_json(nested_json, exclude=['']):
    """Flatten json object with nested keys into a single level.
        Args:
            nested_json: A nested json object.
            exclude: Keys to exclude from output.
        Returns:
            The flattened json object if successful, None otherwise.
    """
    out = {}

    def flatten(x, name='', exclude=exclude):
        if type(x) is dict:
            for a in x:
                if a not in exclude: flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(nested_json)
    return out

## https://stackoverflow.com/questions/52795561/flattening-nested-json-in-pandas-data-frame

In [ ]:
final_df = pd.DataFrame([flatten_json(x) for x in remove_hits_nest['data']])
final_df.head()

In [ ]:
len(final_df)

In [ ]:
## Export file to excel, without the Pandas index, but with the header

final_df.to_excel("outputs/final_df.xlsx", index=False, header=True)

In [ ]:
# ## Remove NESTS manually

# remove_0_nest = pd.concat([remove_hits_nest.drop(['data'], axis=1), remove_hits_nest['data'].apply(pd.Series)], axis=1, join="outer")

# ## REMOVE PIDS NESTS
# remove_pids_nest_1 = pd.concat([remove_0_nest.drop(['pids'], axis=1), remove_0_nest['pids'].apply(pd.Series)], axis=1, join="outer")
# remove_pids_nest_1.rename(columns = {'oai':'pids.oai'}, inplace = True)

# remove_pids_nest_2 = pd.concat([remove_pids_nest_1.drop(['pids.oai'], axis=1), remove_pids_nest_1['pids.oai'].apply(pd.Series)], axis=1, join="outer")
# remove_pids_nest_2.rename(columns = {'identifier':'pids.oai.identifer', 'provider': 'pids.oai.provider'}, inplace = True)


# ## REMOVE PARENT NESTS
# remove_parent_nest_1 = pd.concat([remove_pids_nest_2.drop(['parent'], axis=1), remove_pids_nest_2['parent'].apply(pd.Series)], axis=1, join="outer")
# remove_parent_nest_1.rename(columns = {'id':'parent.id', 'communities':'parent.communities'}, inplace = True)

# remove_parent_nest_2 = pd.concat([remove_parent_nest_1.drop(['parent.communities'], axis=1), remove_parent_nest_1['parent.communities'].apply(pd.Series)], axis=1, join="outer")
# remove_parent_nest_2.rename(columns = {'ids':'parent.communities.ids', 'default':'parent.communities.default'}, inplace = True)

# remove_access_nest_1 = pd.concat([remove_parent_nest_1.drop(['access'], axis=1), remove_parent_nest_1['access'].apply(pd.Series)], axis=1, join="outer")
# remove_access_nest_1.rename(columns = {'status':'access.status', 'files':'access.files', 'embargo':'access.embargo', 'record':'access.record'}, inplace = True)


# remove_parent_nest_2.head()